In [12]:
from sklearn.naive_bayes import MultinomialNB # ideal für counting features wie bow oder tfidf https://towardsdatascience.com/why-how-to-use-the-naive-bayes-algorithms-in-a-regulated-industry-with-sklearn-python-code-dbd8304ab2cf
from sklearn.naive_bayes import GaussianNB # für Features in Decimal Form geeignet
from sklearn.naive_bayes import ComplementNB # ähnlich wie Multinomial, soll sich aber besser für imbalanced data eignen
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    f1_score,
    recall_score,
    precision_score,
    classification_report,
    matthews_corrcoef, 
)
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
import joblib

In [11]:
def evaluate(y_test,y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    mcc = matthews_corrcoef(y_test, y_pred)
    print("F1 Score:", f1)
    print("Recall:", recall)
    print("Precision:", precision)
    print("Accuracy:", accuracy)
    print("MCC:", mcc)
    print("\n")
    print("Confusion Matrix")
    print(pd.DataFrame(confusion_matrix(y_test, y_pred)))
    print("\n")
    print("Classification Report")
    print(classification_report(y_test, y_pred))

## Evaluation neue Vectorize-Funktionen (08.12.)

In [3]:
%run ../../../functions/vectorize_functions.py

In [4]:
filepath_name = (('../../../../data/twitter_hate-speech/train_cleaned.csv'))
df_cleaned = pd.read_csv(filepath_name, encoding='utf-8')

In [6]:
X_train, X_test, y_train, y_test, vectorizer_glv = vectorize_glove(df=df_cleaned, text_column='tweet_cleaned', 
                                                                                 label_column="label")

In [7]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(type(X_train))

(14124, 100)
(6054, 100)
(14124,)
(6054,)
<class 'numpy.ndarray'>


In [8]:
# save vectorizer

joblib.dump(vectorizer_glv, 'joblib_models/vectorizer/vectorizer_glv.joblib')

['joblib_models/vectorizer/vectorizer_glv.joblib']

## GaussianNB

In [9]:
clf = GaussianNB()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

In [13]:
evaluate(y_test, y_pred)

F1 Score: 0.4454901960784314
Recall: 0.773841961852861
Precision: 0.31277533039647576
Accuracy: 0.883217707300958
MCC: 0.4438560072250936


Confusion Matrix
      0    1
0  5063  624
1    83  284


Classification Report
              precision    recall  f1-score   support

           0       0.98      0.89      0.93      5687
           1       0.31      0.77      0.45       367

    accuracy                           0.88      6054
   macro avg       0.65      0.83      0.69      6054
weighted avg       0.94      0.88      0.91      6054



In [14]:
results_list = []

param_grid = {
    'priors' : [None,[.1,.9], [.25,.75], [.5,.5], [.75,.25],[.9,.1]],
    'var_smoothing' : [1.0e-10,1e-9, 0.01, 0.05, 0.1, 0.2, 0.4, 0.5, 0.6, 0.8, 1]
}

nb = GaussianNB()

grid_search = GridSearchCV(estimator=nb, param_grid=param_grid, scoring='f1', cv=3, verbose=2, n_jobs=-1)

grid_search.fit(X_train, y_train)

for idx, params in enumerate(grid_search.cv_results_['params']):
    model = GaussianNB(**params)  
    model.fit(X_train, y_train)  

    y_train_pred = model.predict(X_train)

    y_test_pred = model.predict(X_test)

    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_recall = recall_score(y_train, y_train_pred)
    train_precision = precision_score(y_train, y_train_pred)
    train_f1 = f1_score(y_train, y_train_pred)
    train_mcc = matthews_corrcoef(y_train, y_train_pred)

    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_recall = recall_score(y_test, y_test_pred)
    test_precision = precision_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred)
    test_mcc = matthews_corrcoef(y_test, y_test_pred)

    result_dict = {
        'model': 'Glove',
        'priors': params['priors'],
        'var_smoothing': params['var_smoothing'],
        'train_f1': train_f1,
        'train_recall': train_recall,
        'train_precision': train_precision,
        'train_accuracy': train_accuracy,
        'train_mcc': train_mcc,
        'test_f1': test_f1,
        'test_recall': test_recall,
        'test_precision': test_precision,
        'test_accuracy': test_accuracy,
        'test_mcc': test_mcc
    }

    results_list.append(result_dict)

    filename = f'joblib_models/nb_glv/model_nb_glv_{idx}.joblib'
    joblib.dump(model, filename)

results_df = pd.DataFrame(results_list)

results_df.to_csv('eval_data/nb_glv.csv', index=False)

print(results_df)


Fitting 3 folds for each of 66 candidates, totalling 198 fits
    model      priors  var_smoothing  train_f1  train_recall  train_precision  \
0   Glove        None   1.000000e-10  0.466472      0.746888         0.339143   
1   Glove        None   1.000000e-09  0.466472      0.746888         0.339143   
2   Glove        None   1.000000e-02  0.458558      0.748963         0.330435   
3   Glove        None   5.000000e-02  0.453149      0.750000         0.324652   
4   Glove        None   1.000000e-01  0.448320      0.719917         0.325516   
..    ...         ...            ...       ...           ...              ...   
61  Glove  [0.9, 0.1]   4.000000e-01  0.462882      0.549793         0.399698   
62  Glove  [0.9, 0.1]   5.000000e-01  0.456226      0.486515         0.429487   
63  Glove  [0.9, 0.1]   6.000000e-01  0.434218      0.415975         0.454134   
64  Glove  [0.9, 0.1]   8.000000e-01  0.367801      0.291494         0.498227   
65  Glove  [0.9, 0.1]   1.000000e+00  0.270561 

In [15]:
results_df.sort_values(by=['test_f1'],ascending=False)

,model,priors,var_smoothing,train_f1,train_recall,train_precision,train_accuracy,train_mcc,test_f1,test_recall,test_precision,test_accuracy,test_mcc
5,Glove,None,2.000000e-01,0.457270,0.641079,0.355377,0.896134,0.426641,0.455300,0.673025,0.344011,0.902379,0.435619
61,Glove,"[0.9, 0.1]",4.000000e-01,0.462882,0.549793,0.399698,0.912914,0.423062,0.446786,0.577657,0.364261,0.913280,0.414969
6,Glove,None,4.000000e-01,0.453740,0.478216,0.431648,0.921410,0.412146,0.446363,0.493188,0.407658,0.925834,0.409122
0,Glove,None,1.000000e-10,0.466472,0.746888,0.339143,0.883390,0.451805,0.445490,0.773842,0.312775,0.883218,0.443856
1,Glove,None,1.000000e-09,0.466472,0.746888,0.339143,0.883390,0.451805,0.445490,0.773842,0.312775,0.883218,0.443856
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,Glove,None,1.000000e+00,0.179620,0.107884,0.536082,0.932739,0.218930,0.146789,0.087193,0.463768,0.938553,0.181394
18,Glove,"[0.1, 0.9]",5.000000e-01,0.156222,0.991701,0.084789,0.268833,0.130453,0.144119,1.000000,0.077656,0.279980,0.134662
19,Glove,"[0.1, 0.9]",6.000000e-01,0.147607,0.994813,0.079717,0.215803,0.108968,0.134088,1.000000,0.071862,0.217047,0.109391
20,Glove,"[0.1, 0.9]",8.000000e-01,0.136881,0.997925,0.073480,0.141037,0.073831,0.123113,1.000000,0.065594,0.136439,0.072761
